# Teil 2 der Semesterarbeit: Parallel- und Zentralprojektionen

Deadline: Mittwoch,  21.10.2020

### 1. Parallelprojektion: https://www.youtube.com/watch?v=FXgTfIgZZZg

Achtung:  Parallelepiped (siehe wiki)

### 2. Zenralprojektion -> siehe Buch

## Theorie

Im Theorieteil müssen die mathematischen Grundlagen beschrieben werden, insbesondere die Idee und Anwendung der homogenen Koordinaten.


Aufgaben:
Sie verstehen die Matrizendarstellungen von Parallelprojektionen und Zentralprojektionen.
Sie können Zentralprojektionen mit numpy berechnen und mit matplotlib visualisieren.

Schreiben Sie eine Python Klasse für ein Parallelepiped (Spat).

Erstellen Sie eine Pythonfunktion, um ein Parallelepiped mit einer Paralleprojektion  auf der 𝑥𝑦
x
y
-Ebene darzustellen.
Die Funktion soll als Argumente ein Parallelepiped und einen Richtungsvektor verwenden; die Funktion sollte eine Matplotlib-Graphik erstellen.
Erstellen Sie eine Pythonfunktion, um ein Parallelepiped mit einer Zenralprojektion  auf der 𝑥𝑦
x
y
-Ebene darzustellen.
Die Funktion soll als Argumente ein Parallelepiped und ein Projektionszentrum (Kamera) verwenden; die Funktion sollte eine Matplotlib-Graphik erstellen.
Hilfsmittel: Numpy, Matplotlib, Jupyter Notebook (oder Python-IDE).